In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [4]:
df=pd.read_excel("Text_Mining_Reviews.xlsx",sheet_name="Data")

In [5]:
df

,Number,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score
0,0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,682.0,37.44,negative
1,1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,712.0,31.82,positive
2,2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,538.0,26.21,negative
3,3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,1354.0,59.90,negative
4,4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,2331.0,119.50,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,3454.0,167.86,NaN
16655,16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,1456.0,73.63,NaN
16656,16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,872.0,41.18,NaN
16657,16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,877.0,41.50,NaN


In [6]:
df=df.drop(['Number'], axis=1)

In [7]:
df.index

RangeIndex(start=0, stop=16659, step=1)

In [8]:
df.iloc[0][0]+" "+df.iloc[0][3]

'Surprise from overpayment I took a loan of 100,000 rubles in 2015.\xa0The office immediately began to breed for additional services, insurance, etc.\xa0Under the terms, they lied absolutely everything.\xa0When I was about to leave, the conditions softened.\xa0As a result, after a year of payments on schedule, I owed 92,000 rubles.\xa0Another year passed and I boldly assumed that the balance was not more than 70,000 on the loan.\xa0As I literally put it, I was surprised when, after another year, I owed less on the main debt of only 1,000 rubles less, totaling 91,000 rubles!\xa0That is, for 24 months of payments in the amount of at least 85,000 rubles, less than 9,000 rubles were written off from the main debt.\xa0I have not seen such predatory movements in any bank.\xa0In no case do not contact them.'

In [9]:
l=[]
for i in range(0,len(df.index)):
    s=str(df.iloc[i][0])+" "+str(df.iloc[i][3])
    l.append(s)

In [10]:
l[0]

'Surprise from overpayment I took a loan of 100,000 rubles in 2015.\xa0The office immediately began to breed for additional services, insurance, etc.\xa0Under the terms, they lied absolutely everything.\xa0When I was about to leave, the conditions softened.\xa0As a result, after a year of payments on schedule, I owed 92,000 rubles.\xa0Another year passed and I boldly assumed that the balance was not more than 70,000 on the loan.\xa0As I literally put it, I was surprised when, after another year, I owed less on the main debt of only 1,000 rubles less, totaling 91,000 rubles!\xa0That is, for 24 months of payments in the amount of at least 85,000 rubles, less than 9,000 rubles were written off from the main debt.\xa0I have not seen such predatory movements in any bank.\xa0In no case do not contact them.'

In [11]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
tokenizer=RegexpTokenizer(r'\w+')
en_stopwords=set(stopwords.words('english'))
ps=PorterStemmer()

In [13]:
import re
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [14]:
def getCleanedText(text):
    text=str(text)
    text=text.lower()
    text = re.sub('[^A-Za-z]', ' ', text)
#     text=[x for x in text if x.lower() in words]
    tokens=tokenizer.tokenize(text)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    stemmed_tokens=[ps.stem(tokens) for tokens in new_tokens]
    final=[x for x in stemmed_tokens if x.isdigit()==False ]
    final=[x for x in final if x.lower() in words and len(x)>2]
    clean_text=" ".join(final)
    return clean_text

In [15]:
getCleanedText(l[0])
l=[getCleanedText(i) for i in l]

In [16]:
len(l)

16659

In [17]:
df['clean_text']=l

In [18]:
df

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score,clean_text
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,682.0,37.44,negative,overpay took loan breed term lie soften result...
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,712.0,31.82,positive,great card year card interest free period mont...
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,538.0,26.21,negative,call third would like ask question right bank ...
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,1354.0,59.90,negative,claim connect loan amount bank contract way co...
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,2331.0,119.50,negative,credit fraud husband need money went nearest s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,3454.0,167.86,NaN,take care money cooper bank seem good even per...
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,1456.0,73.63,NaN,card lock may made sever transact amount sever...
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,872.0,41.18,NaN,good afternoon name client bank moment delay d...
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,877.0,41.50,NaN,mislead enter loan agreement loan bank told lo...


In [19]:
df.iloc[0][13]

'overpay took loan breed term lie soften result year payment owe year pass loan liter put year owe less main debt less total month payment amount least less written main debt seen movement bank case contact'

In [20]:
df2=df[['clean_text','binary_score']]

In [21]:
df2

,clean_text,binary_score
0,overpay took loan breed term lie soften result...,negative
1,great card year card interest free period mont...,positive
2,call third would like ask question right bank ...,negative
3,claim connect loan amount bank contract way co...,negative
4,credit fraud husband need money went nearest s...,negative
...,...,...
16654,take care money cooper bank seem good even per...,NaN
16655,card lock may made sever transact amount sever...,NaN
16656,good afternoon name client bank moment delay d...,NaN
16657,mislead enter loan agreement loan bank told lo...,NaN


In [22]:
df2=df2.dropna()
df2 = df2.drop(df2.index[3734])
df2 = df2.drop(df2.index[4507])
df2 = df2.drop(df2.index[6610])
df2 = df2.drop(df2.index[8543])
df2 = df2.drop(df2.index[9150])
df2 = df2.drop(df2.index[9969])
df2 = df2.drop(df2.index[10287])
df2 = df2.drop(df2.index[11376])
df2 = df2.drop(df2.index[15277])

In [23]:
df2

,clean_text,binary_score
0,overpay took loan breed term lie soften result...,negative
1,great card year card interest free period mont...,positive
2,call third would like ask question right bank ...,negative
3,claim connect loan amount bank contract way co...,negative
4,credit fraud husband need money went nearest s...,negative
...,...,...
16646,last payment made overpay three year underpay ...,negative
16647,note want write disgust summer bank friend tak...,negative
16648,term contract claim agreement sign credit card...,negative
16649,give loan term take bad even dare give assess ...,negative


In [24]:
df2 = df2.reset_index(drop=True)
df2["binary_score"]=df2["binary_score"].replace({"positive":1,"negative":0})

In [25]:
df2

,clean_text,binary_score
0,overpay took loan breed term lie soften result...,0
1,great card year card interest free period mont...,1
2,call third would like ask question right bank ...,0
3,claim connect loan amount bank contract way co...,0
4,credit fraud husband need money went nearest s...,0
...,...,...
16637,last payment made overpay three year underpay ...,0
16638,note want write disgust summer bank friend tak...,0
16639,term contract claim agreement sign credit card...,0
16640,give loan term take bad even dare give assess ...,0


In [26]:
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def get_polarity(text):
    return TextBlob(text).sentiment.polarity


In [27]:
df['subjectivity']=df['clean_text'].apply(get_subjectivity)
df['polarity']=df['clean_text'].apply(get_polarity)


In [28]:
df

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score,clean_text,subjectivity,polarity
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,682.0,37.44,negative,overpay took loan breed term lie soften result...,0.288095,-0.066667
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,712.0,31.82,positive,great card year card interest free period mont...,0.631250,0.466667
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,538.0,26.21,negative,call third would like ask question right bank ...,0.267143,0.097143
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,1354.0,59.90,negative,claim connect loan amount bank contract way co...,0.100000,0.300000
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,2331.0,119.50,negative,credit fraud husband need money went nearest s...,0.406944,-0.016865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,3454.0,167.86,NaN,take care money cooper bank seem good even per...,0.386806,0.073264
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,1456.0,73.63,NaN,card lock may made sever transact amount sever...,0.434972,-0.070223
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,872.0,41.18,NaN,good afternoon name client bank moment delay d...,0.550000,0.418182
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,877.0,41.50,NaN,mislead enter loan agreement loan bank told lo...,0.500000,0.416667


In [29]:
df.columns

Index(['responses_header', 'responses_rating_grade', 'responses_status',
       'responses_message', 'responses_datetime', 'reviewer', 'comments',
       'views', 'id', 'have_email', 'responses_message_length',
       'gunning_fog_index', 'binary_score', 'clean_text', 'subjectivity',
       'polarity'],
      dtype='object')

In [30]:
def getSIA(text):
    sia=SentimentIntensityAnalyzer()
    sentiment=sia.polarity_scores(text)
    return sentiment

In [31]:
compound=[]
neg=[]
pos=[]
neu=[]
SIA=0
for i in range(0,len(df['clean_text'])):
    SIA=getSIA(df['clean_text'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    pos.append(SIA['pos'])
    neu.append(SIA['neu'])

In [32]:
df['compound']=compound
df['negative']=neg
df['positive']=pos
df['neutral']=neu

In [33]:
df

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score,clean_text,subjectivity,polarity,compound,negative,positive,neutral
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,682.0,37.44,negative,overpay took loan breed term lie soften result...,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,712.0,31.82,positive,great card year card interest free period mont...,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,538.0,26.21,negative,call third would like ask question right bank ...,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,1354.0,59.90,negative,claim connect loan amount bank contract way co...,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,2331.0,119.50,negative,credit fraud husband need money went nearest s...,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,3454.0,167.86,NaN,take care money cooper bank seem good even per...,0.386806,0.073264,0.9758,0.069,0.176,0.755
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,1456.0,73.63,NaN,card lock may made sever transact amount sever...,0.434972,-0.070223,-0.7234,0.187,0.146,0.667
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,872.0,41.18,NaN,good afternoon name client bank moment delay d...,0.550000,0.418182,-0.4464,0.185,0.094,0.722
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,877.0,41.50,NaN,mislead enter loan agreement loan bank told lo...,0.500000,0.416667,0.7964,0.057,0.227,0.716


In [34]:
df.columns

Index(['responses_header', 'responses_rating_grade', 'responses_status',
       'responses_message', 'responses_datetime', 'reviewer', 'comments',
       'views', 'id', 'have_email', 'responses_message_length',
       'gunning_fog_index', 'binary_score', 'clean_text', 'subjectivity',
       'polarity', 'compound', 'negative', 'positive', 'neutral'],
      dtype='object')

In [35]:
dx=df[['comments','views', 'id', 'have_email', 'responses_message_length','gunning_fog_index', 'binary_score', 'subjectivity','polarity','compound', 'negative', 'positive', 'neutral']]

In [36]:
dx

,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,positive,neutral
0,0,1169,21.0,1.0,682.0,37.44,negative,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,0,1072,117.0,0.0,712.0,31.82,positive,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,2,1280,131.0,0.0,538.0,26.21,negative,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,5,1085,156.0,0.0,1354.0,59.90,negative,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,1,1556,172.0,0.0,2331.0,119.50,negative,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,0,1454,554833.0,0.0,3454.0,167.86,NaN,0.386806,0.073264,0.9758,0.069,0.176,0.755
16655,4,1432,554880.0,0.0,1456.0,73.63,NaN,0.434972,-0.070223,-0.7234,0.187,0.146,0.667
16656,1,1280,554956.0,0.0,872.0,41.18,NaN,0.550000,0.418182,-0.4464,0.185,0.094,0.722
16657,2,1354,554957.0,0.0,877.0,41.50,NaN,0.500000,0.416667,0.7964,0.057,0.227,0.716


In [37]:
dx["binary_score"]=dx["binary_score"].replace({"positive":1,"negative":0})

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
dx

,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,positive,neutral
0,0,1169,21.0,1.0,682.0,37.44,0.0,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,0,1072,117.0,0.0,712.0,31.82,1.0,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,2,1280,131.0,0.0,538.0,26.21,0.0,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,5,1085,156.0,0.0,1354.0,59.90,0.0,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,1,1556,172.0,0.0,2331.0,119.50,0.0,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,0,1454,554833.0,0.0,3454.0,167.86,NaN,0.386806,0.073264,0.9758,0.069,0.176,0.755
16655,4,1432,554880.0,0.0,1456.0,73.63,NaN,0.434972,-0.070223,-0.7234,0.187,0.146,0.667
16656,1,1280,554956.0,0.0,872.0,41.18,NaN,0.550000,0.418182,-0.4464,0.185,0.094,0.722
16657,2,1354,554957.0,0.0,877.0,41.50,NaN,0.500000,0.416667,0.7964,0.057,0.227,0.716


In [39]:
dx.isnull().sum()

comments                    0
views                       0
id                          9
have_email                  9
responses_message_length    9
gunning_fog_index           9
binary_score                8
subjectivity                0
polarity                    0
compound                    0
negative                    0
positive                    0
neutral                     0
dtype: int64

In [40]:
dx=dx.dropna()
dx = dx.drop(dx.index[3734])
dx = dx.drop(dx.index[4507])
dx = dx.drop(dx.index[6610])
dx = dx.drop(dx.index[8543])
dx = dx.drop(dx.index[9150])
dx = dx.drop(dx.index[9969])
dx = dx.drop(dx.index[10287])
dx = dx.drop(dx.index[11376])
dx = dx.drop(dx.index[15277])

In [41]:
dx.isnull().sum()

comments                    0
views                       0
id                          0
have_email                  0
responses_message_length    0
gunning_fog_index           0
binary_score                0
subjectivity                0
polarity                    0
compound                    0
negative                    0
positive                    0
neutral                     0
dtype: int64

In [42]:
dx.shape

(16633, 13)

In [43]:
dx=dx.drop(['id'],axis=1)

In [44]:
dx

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,positive,neutral
0,0,1169,1.0,682.0,37.44,0.0,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,0,1072,0.0,712.0,31.82,1.0,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,2,1280,0.0,538.0,26.21,0.0,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,5,1085,0.0,1354.0,59.90,0.0,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,1,1556,0.0,2331.0,119.50,0.0,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...
16646,2,1937,0.0,1289.0,67.70,0.0,0.176190,-0.078571,0.2500,0.061,0.079,0.860
16647,0,1319,0.0,1086.0,48.36,0.0,0.384341,0.010806,0.6249,0.126,0.203,0.671
16648,1,1329,0.0,1132.0,50.23,0.0,0.400000,0.200000,0.9477,0.000,0.255,0.745
16649,0,1447,0.0,1156.0,54.29,0.0,0.537626,0.041667,0.8369,0.091,0.198,0.712


In [45]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
col=['comments','views','have_email','responses_message_length','gunning_fog_index']
dx[['comments','views','have_email','responses_message_length','gunning_fog_index']] = pd.DataFrame(scaler.fit_transform(dx[['comments','views','have_email','responses_message_length','gunning_fog_index']]))

dx

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,positive,neutral
0,0.000000,0.027590,1.0,0.041849,0.051988,0.0,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,0.000000,0.024992,0.0,0.043935,0.043551,1.0,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,0.041667,0.030564,0.0,0.031839,0.035129,0.0,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,0.104167,0.025340,0.0,0.088564,0.085705,0.0,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,0.020833,0.037957,0.0,0.156482,0.175179,0.0,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...
16646,NaN,NaN,NaN,NaN,NaN,0.0,0.176190,-0.078571,0.2500,0.061,0.079,0.860
16647,NaN,NaN,NaN,NaN,NaN,0.0,0.384341,0.010806,0.6249,0.126,0.203,0.671
16648,NaN,NaN,NaN,NaN,NaN,0.0,0.400000,0.200000,0.9477,0.000,0.255,0.745
16649,NaN,NaN,NaN,NaN,NaN,0.0,0.537626,0.041667,0.8369,0.091,0.198,0.712


In [46]:
dx = dx.reset_index(drop=True)

In [47]:
dx.isnull().sum()

comments                    18
views                       18
have_email                  18
responses_message_length    18
gunning_fog_index           18
binary_score                 0
subjectivity                 0
polarity                     0
compound                     0
negative                     0
positive                     0
neutral                      0
dtype: int64

In [48]:
dx=dx.dropna()


In [49]:
dx

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,positive,neutral
0,0.000000,0.027590,1.0,0.041849,0.051988,0.0,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,0.000000,0.024992,0.0,0.043935,0.043551,1.0,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,0.041667,0.030564,0.0,0.031839,0.035129,0.0,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,0.104167,0.025340,0.0,0.088564,0.085705,0.0,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,0.020833,0.037957,0.0,0.156482,0.175179,0.0,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...
16610,0.041667,0.048162,0.0,0.084046,0.097415,0.0,0.400000,-0.300000,-0.6597,0.178,0.000,0.822
16611,0.000000,0.031608,0.0,0.069934,0.068381,0.0,0.326531,0.152721,0.9719,0.000,0.233,0.767
16612,0.020833,0.031876,0.0,0.073132,0.071188,0.0,0.368110,0.117232,0.9821,0.066,0.216,0.718
16613,0.000000,0.035037,0.0,0.074800,0.077283,0.0,0.364286,-0.121429,-0.6414,0.163,0.097,0.740


In [50]:
dx.binary_score = dx.binary_score.astype('int64')

In [51]:
dx

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,positive,neutral
0,0.000000,0.027590,1.0,0.041849,0.051988,0,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,0.000000,0.024992,0.0,0.043935,0.043551,1,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,0.041667,0.030564,0.0,0.031839,0.035129,0,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,0.104167,0.025340,0.0,0.088564,0.085705,0,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,0.020833,0.037957,0.0,0.156482,0.175179,0,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...
16610,0.041667,0.048162,0.0,0.084046,0.097415,0,0.400000,-0.300000,-0.6597,0.178,0.000,0.822
16611,0.000000,0.031608,0.0,0.069934,0.068381,0,0.326531,0.152721,0.9719,0.000,0.233,0.767
16612,0.020833,0.031876,0.0,0.073132,0.071188,0,0.368110,0.117232,0.9821,0.066,0.216,0.718
16613,0.000000,0.035037,0.0,0.074800,0.077283,0,0.364286,-0.121429,-0.6414,0.163,0.097,0.740


In [52]:
# y=dx['binary_score']
# x=dx.drop(['binary_score'],axis=1)

In [53]:
X=dx
X=np.array(X.drop(['binary_score'],1))
y=np.array(dx['binary_score'])

In [54]:
y

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [55]:
X

array([[0.        , 0.02759027, 1.        , ..., 0.116     , 0.        ,
        0.884     ],
       [0.        , 0.02499196, 0.        , ..., 0.084     , 0.457     ,
        0.459     ],
       [0.04166667, 0.03056359, 0.        , ..., 0.165     , 0.062     ,
        0.773     ],
       ...,
       [0.02083333, 0.03187614, 0.        , ..., 0.066     , 0.216     ,
        0.718     ],
       [0.        , 0.03503697, 0.        , ..., 0.163     , 0.097     ,
        0.74      ],
       [0.        , 0.03217079, 0.        , ..., 0.073     , 0.287     ,
        0.64      ]])

In [56]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [57]:
model=LinearDiscriminantAnalysis().fit(x_train,y_train)

In [58]:
y_pred=model.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [59]:
y_test

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [60]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.95      0.79      2186
           1       0.51      0.10      0.17      1137

    accuracy                           0.66      3323
   macro avg       0.59      0.53      0.48      3323
weighted avg       0.61      0.66      0.58      3323



In [61]:
dx.columns

Index(['comments', 'views', 'have_email', 'responses_message_length',
       'gunning_fog_index', 'binary_score', 'subjectivity', 'polarity',
       'compound', 'negative', 'positive', 'neutral'],
      dtype='object')

In [62]:
x2=dx[['comments', 'views', 'have_email','gunning_fog_index','subjectivity', 'polarity','compound', 'negative', 'positive', 'neutral']]
X2=np.array(x2)
y2=np.array(dx['binary_score'])

In [63]:
x2_train,x2_test,y2_train,y2_test=train_test_split(X2,y2,test_size=0.2,random_state=0)

In [64]:
model2=LinearDiscriminantAnalysis().fit(x2_train,y2_train)

In [65]:
y_pred2=model2.predict(x2_test)
y_pred2

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [66]:
print(classification_report(y2_test,y_pred2))

              precision    recall  f1-score   support

           0       0.67      0.95      0.79      2186
           1       0.52      0.10      0.17      1137

    accuracy                           0.66      3323
   macro avg       0.59      0.53      0.48      3323
weighted avg       0.62      0.66      0.58      3323



In [69]:
from sklearn.ensemble import AdaBoostClassifier
import sklearn.metrics as metrics

In [70]:
a_clf = AdaBoostClassifier().fit(x_train,y_train)
ypred = a_clf.predict(x_test)
print("AdaBoostClassifier accuracy:", metrics.accuracy_score(y_test, ypred))

AdaBoostClassifier accuracy: 0.655732771591935


In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [72]:
rand_clf = RandomForestClassifier(criterion= 'gini',
 max_depth = 17,
 max_features = 'auto',
 min_samples_leaf = 1,
 min_samples_split= 2,
 n_estimators = 90,random_state=6)

In [75]:
model_rf=rand_clf.fit(x_train,y_train)

In [74]:
rand_clf.score(x_test,y_test)

0.6704784832982245

In [76]:
y_pred_new = model_rf.predict(x_test)
predictions_new = [round(value) for value in y_pred_new]
accuracy_new = accuracy_score(y_test,predictions_new)
accuracy_new

0.6704784832982245

In [77]:
print(classification_report(y_test,y_pred_new))

              precision    recall  f1-score   support

           0       0.69      0.91      0.78      2186
           1       0.55      0.21      0.31      1137

    accuracy                           0.67      3323
   macro avg       0.62      0.56      0.55      3323
weighted avg       0.64      0.67      0.62      3323



In [134]:
model_rf_2=rand_clf.fit(x2_train,y2_train)

In [135]:
y_pred_new_rf = model_rf.predict(x_test)
predictions_new_rf = [round(value) for value in y_pred_new_rf]
accuracy_new_rf = accuracy_score(y_test,predictions_new_rf)
accuracy_new_rf

0.6731868793259104

In [136]:
print(classification_report(y_test,y_pred_new_rf))

              precision    recall  f1-score   support

           0       0.68      0.94      0.79      2186
           1       0.58      0.16      0.26      1137

    accuracy                           0.67      3323
   macro avg       0.63      0.55      0.52      3323
weighted avg       0.65      0.67      0.61      3323



In [78]:
from sklearn.linear_model import SGDClassifier
import sklearn.metrics as metrics

In [79]:
s_clf = SGDClassifier().fit(x_train,y_train)
ypred = s_clf.predict(x_test)
print("SGDClassifier accuracy:", metrics.accuracy_score(y_test, ypred))

SGDClassifier accuracy: 0.6578393018356906


In [80]:
print(classification_report(y_test,ypred))

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      2186
           1       0.00      0.00      0.00      1137

    accuracy                           0.66      3323
   macro avg       0.33      0.50      0.40      3323
weighted avg       0.43      0.66      0.52      3323



C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [87]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [88]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_model=knn.fit(x_train,y_train)
knn.score(x_test,y_test)

0.6030695154980439

In [91]:
knn_model_predict=knn_model.predict(x_test)
knn_model_predict = [round(value) for value in knn_model_predict]
accuracy_new2 = accuracy_score(y_test,knn_model_predict)
accuracy_new2

0.6030695154980439

In [92]:
print(classification_report(y_test,knn_model_predict))

              precision    recall  f1-score   support

           0       0.68      0.75      0.71      2186
           1       0.40      0.32      0.35      1137

    accuracy                           0.60      3323
   macro avg       0.54      0.53      0.53      3323
weighted avg       0.58      0.60      0.59      3323



In [94]:
bag_knn = BaggingClassifier(KNeighborsClassifier(n_neighbors=5),
                            n_estimators=10, max_samples=0.5,
                            bootstrap=True, random_state=3,oob_score=True) 

In [95]:
bag_knn_model=bag_knn.fit(x_train,y_train)
bag_knn_model.score(x_test,y_test)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\USER\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:645: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


0.6256394823954258

In [96]:
bag_knn_pred=bag_knn_model.predict(x_test)
bag_knn_model_predict = [round(value) for value in bag_knn_pred]
accuracy_new3 = accuracy_score(y_test,bag_knn_model_predict)
accuracy_new3

0.6256394823954258

In [97]:
print(classification_report(y_test,bag_knn_model_predict))

              precision    recall  f1-score   support

           0       0.67      0.84      0.75      2186
           1       0.41      0.22      0.29      1137

    accuracy                           0.63      3323
   macro avg       0.54      0.53      0.52      3323
weighted avg       0.58      0.63      0.59      3323



In [98]:
pasting_knn = BaggingClassifier(KNeighborsClassifier(n_neighbors=5),
                            n_estimators=10, max_samples=0.5,
                            bootstrap=False, random_state=3) 

In [99]:
pasting_knn_model=pasting_knn.fit(x_train,y_train)
pasting_knn.score(x_test,y_test)

0.6268432139632862

In [100]:
pasting_knn_pred=pasting_knn_model.predict(x_test)
pasting_knn_model_predict = [round(value) for value in pasting_knn_pred]
accuracy_new4 = accuracy_score(y_test,pasting_knn_model_predict)
accuracy_new4

0.6268432139632862

In [101]:
print(classification_report(y_test,pasting_knn_model_predict))

              precision    recall  f1-score   support

           0       0.68      0.83      0.75      2186
           1       0.42      0.23      0.30      1137

    accuracy                           0.63      3323
   macro avg       0.55      0.53      0.52      3323
weighted avg       0.59      0.63      0.59      3323



In [104]:
import xgboost as xgb
import pickle
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [105]:
XGB_model = XGBClassifier(objective='binary:logistic')
XGB_model.fit(x_train,y_train)

C:\Users\USER\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:02:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [107]:
# cheking initial train accuracy
XGB_y_pred = XGB_model.predict(x_train)
predictions_1 = [round(value) for value in XGB_y_pred]
accuracy_1 = accuracy_score(y_train,predictions_1)
accuracy_1

0.8649563647306651

In [108]:
# cheking initial test accuracy
XGB_y_pred_2 = XGB_model.predict(x_test)
predictions_2 = [round(value) for value in XGB_y_pred_2]
accuracy_2 = accuracy_score(y_test,predictions_2)
accuracy_2

0.6536262413481794

In [109]:
print(classification_report(y_test,predictions_2))

              precision    recall  f1-score   support

           0       0.69      0.85      0.76      2186
           1       0.49      0.27      0.35      1137

    accuracy                           0.65      3323
   macro avg       0.59      0.56      0.56      3323
weighted avg       0.62      0.65      0.62      3323



In [112]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(x_train,y_train)

LogisticRegression()

In [113]:
y_pred_log = log_reg.predict(x_test)

In [114]:
print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.67      0.96      0.79      2186
           1       0.52      0.09      0.15      1137

    accuracy                           0.66      3323
   macro avg       0.59      0.52      0.47      3323
weighted avg       0.62      0.66      0.57      3323



In [115]:
print(classification_report(y_test,y_pred_new))

              precision    recall  f1-score   support

           0       0.69      0.91      0.78      2186
           1       0.55      0.21      0.31      1137

    accuracy                           0.67      3323
   macro avg       0.62      0.56      0.55      3323
weighted avg       0.64      0.67      0.62      3323



In [119]:
from sklearn.model_selection import train_test_split,GridSearchCV


In [120]:
grid_param = {
    "n_estimators" : [90],
    'criterion': ['gini'],
    'max_depth' : range(2,20,3),
    'min_samples_leaf' : range(1,10,3),
    'min_samples_split': range(2,10,3),
    'max_features' : ['auto','log2']
}

In [121]:
grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=3,n_jobs =1,verbose = 2)

In [123]:
grid_search.fit(x_train,y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90; total time=   0.3s
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90; total time=   0.3s
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=90; total time=   0.3s
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=90; total time=   0.3s
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=90; total time=   0.3s
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=90; total time=   0.3s
[CV] END criterion=gini, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=90; total time=   0.3s

[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=90; total time=   0.6s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=8, n_estimators=90; total time=   0.6s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=90; total time=   1.1s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=90; total time=   1.3s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=90; total time=   0.6s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=90; total time=   0.6s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=90; total time=   0.6s
[CV] END criterion=gini, max_depth=5, max_features=auto, min_s

[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=4, min_samples_split=8, n_estimators=90; total time=   1.8s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=4, min_samples_split=8, n_estimators=90; total time=   0.9s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=4, min_samples_split=8, n_estimators=90; total time=   0.9s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=7, min_samples_split=2, n_estimators=90; total time=   1.9s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=7, min_samples_split=2, n_estimators=90; total time=   1.1s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=7, min_samples_split=2, n_estimators=90; total time=   0.9s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_samples_leaf=7, min_samples_split=5, n_estimators=90; total time=   1.5s
[CV] END criterion=gini, max_depth=8, max_features=auto, min_s

[CV] END criterion=gini, max_depth=11, max_features=auto, min_samples_leaf=7, min_samples_split=5, n_estimators=90; total time=   1.1s
[CV] END criterion=gini, max_depth=11, max_features=auto, min_samples_leaf=7, min_samples_split=5, n_estimators=90; total time=   1.5s
[CV] END criterion=gini, max_depth=11, max_features=auto, min_samples_leaf=7, min_samples_split=8, n_estimators=90; total time=   1.9s
[CV] END criterion=gini, max_depth=11, max_features=auto, min_samples_leaf=7, min_samples_split=8, n_estimators=90; total time=   1.1s
[CV] END criterion=gini, max_depth=11, max_features=auto, min_samples_leaf=7, min_samples_split=8, n_estimators=90; total time=   1.8s
[CV] END criterion=gini, max_depth=11, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=90; total time=   1.7s
[CV] END criterion=gini, max_depth=11, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=90; total time=   1.2s
[CV] END criterion=gini, max_depth=11, max_features=log

[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=90; total time=   2.3s
[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=90; total time=   1.4s
[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=90; total time=   2.9s
[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=90; total time=   1.7s
[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=8, n_estimators=90; total time=   1.8s
[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=8, n_estimators=90; total time=   2.1s
[CV] END criterion=gini, max_depth=14, max_features=log2, min_samples_leaf=1, min_samples_split=8, n_estimators=90; total time=   1.3s
[CV] END criterion=gini, max_depth=14, max_features=log

[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=90; total time=   1.4s
[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=90; total time=   2.1s
[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=90; total time=   1.9s
[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=90; total time=   1.7s
[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=90; total time=   2.9s
[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=90; total time=   1.4s
[CV] END criterion=gini, max_depth=17, max_features=log2, min_samples_leaf=4, min_samples_split=8, n_estimators=90; total time=   2.4s
[CV] END criterion=gini, max_depth=17, max_features=log

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_depth=17, n_estimators=90,
                                              random_state=6),
             n_jobs=1,
             param_grid={'criterion': ['gini'], 'max_depth': range(2, 20, 3),
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': range(1, 10, 3),
                         'min_samples_split': range(2, 10, 3),
                         'n_estimators': [90]},
             verbose=2)

In [125]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'auto',
 'min_samples_leaf': 7,
 'min_samples_split': 2,
 'n_estimators': 90}

In [126]:
rand_clf_2 = RandomForestClassifier(criterion= 'gini',
 max_depth = 8,
 max_features = 'auto',
 min_samples_leaf = 7,
 min_samples_split= 2,
 n_estimators = 90,random_state=6)

In [129]:
model_rf=rand_clf_2.fit(x_train,y_train)

In [130]:
y_pred_rf = model_rf.predict(x_test)
rf_pred = [round(value) for value in y_pred_rf]
rf_pred_accu = accuracy_score(y_test,rf_pred)
rf_pred_accu

0.6731868793259104

In [131]:
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       0.68      0.94      0.79      2186
           1       0.58      0.16      0.26      1137

    accuracy                           0.67      3323
   macro avg       0.63      0.55      0.52      3323
weighted avg       0.65      0.67      0.61      3323



In [133]:
print(classification_report(y_test,y_pred_new))

              precision    recall  f1-score   support

           0       0.69      0.91      0.78      2186
           1       0.55      0.21      0.31      1137

    accuracy                           0.67      3323
   macro avg       0.62      0.56      0.55      3323
weighted avg       0.64      0.67      0.62      3323

